In [108]:
import yaml
import mock_parser
from pprint import pprint
from sympy import *
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
)

## Load file contents

In [109]:
with open("ret_pen.yml", "r") as config:
    model_config = yaml.safe_load_all(config)
    
    model = [stage for stage in model_config]

## Model metadata

In [110]:
print(model[0]["model"])
print(model[0]["author"])
print(model[0]["affiliation"])
print(model[0]["date"])
pprint(model[0]["abstract"])

Consumption-Pension Deposit Model
Alan Lujan
['Johns Hopkins University', 'Econ-ARK']
12/13/2023
This model is a lifecycle model with two life stages: working life and retirem
ent. The working life stage is further divided into three blocks: expectation,
 deposit, and consumption. During the working life stage, a consumer receives 
permanent and transitory income shocks and  decides how much to consume and ho
w much to deposit in a pension fund. The pension fund is invested in a risky a
sset and the consumer receives a risky return on the pension fund. The consume
r also has a liquid asset which is invested in a risk-free account.


# Stage 1: Working Life before Retirement

In [111]:
print(model[1]["name"])
pprint(model[1]["description"])

working
working life before retirement


In [112]:
model[1]["states"]

[{'name': 'aNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (a)ssets after consumption'},
 {'name': 'bNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (b)alance in pension fund after deposit'},
 {'name': 'jNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized wealth (j)ust before income'},
 {'name': 'kNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized pension (k)apital before income'},
 {'name': 'lNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (l)iquid assets after deposit'},
 {'name': 'mNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (m)arket resources'},
 {'name': 'nNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (n)et retirement balance'}]

In [113]:
model[1]["controls"]

[{'name': 'cNrm',
  'domain': [0.0, 'lNrm'],
  'description': 'normalized consumption'},
 {'name': 'dNrm',
  'domain': [0.0, 'mNrm'],
  'description': 'normalized pension deposit'}]

In [114]:
model[1]["distributions"]

[{'name': 'perm',
  'description': 'permanent income shock',
  'distribution': <mock_parser.MeanOneLogNormal at 0x2c5316b1e40>},
 {'name': 'tran',
  'description': 'transitory income shock',
  'distribution': <mock_parser.MeanOneLogNormal at 0x2c5316b2950>},
 {'name': 'risky',
  'description': 'risky asset return',
  'distribution': <mock_parser.LogNormal at 0x2c5316b2890>}]

## Stage 1: Blocks

In [115]:
[block["name"] for block in model[1]["blocks"]]

['expectation', 'deposit', 'consumption']

### Expectation Block

In [116]:
model[1]["blocks"][0]

{'name': 'expectation',
 'description': 'expectation block',
 'states': [{'name': 'jNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized wealth (j)ust before income'},
  {'name': 'kNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized pension (k)apital before income'}],
 'distributions': [{'name': 'perm',
   'description': 'permanent income shock',
   'distribution': <mock_parser.MeanOneLogNormal at 0x2c5316b1e40>},
  {'name': 'tran',
   'description': 'transitory income shock',
   'distribution': <mock_parser.MeanOneLogNormal at 0x2c5316b2950>},
  {'name': 'risky',
   'description': 'risky asset return',
   'distribution': <mock_parser.LogNormal at 0x2c5316b2890>}],
 'parameters': ['CRRA',
  'Rfree',
  'PermGroFac',
  'std_perm',
  'std_tran',
  'mean_risky',
  'std_risky'],
 'equations': {'objective': ['v(jNrm, kNrm) = PermAdj^(1-CRRA) * w(mNrm, nNrm)'],
  'transitions': ['mNrm = Rfree * jNrm / PermAdj + tran',
   'nNrm = risky * kNrm / PermAdj'],
  'definitions': 

### Deposit Block

In [117]:
deposit_block = model[1]["blocks"][1]
deposit_block

{'name': 'deposit',
 'description': 'deposit block',
 'states': [{'name': 'mNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (m)arket resources'},
  {'name': 'nNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (n)et retirement balance'}],
 'controls': [{'name': 'dNrm',
   'domain': [0.0, 'mNrm'],
   'description': 'normalized pension deposit'}],
 'parameters': ['chi'],
 'equations': {'objective': ['v(mNrm, nNrm) = w(lNrm, bNrm)'],
  'transitions': ['lNrm = mNrm - dNrm', 'bNrm = nNrm + dNrm + g(dNrm)'],
  'definitions': ['g(dNrm) = chi * log(dNrm + 1)']}}

### Consumption Block

In [118]:
consumption_block = model[1]["blocks"][2]

## Endogenous Grid Method

### Deposit Solution

In [119]:

# Parse equations from the dictionary
def parse_eq(eq_str):
    # Remove 'Nrm' from the equation string
    eq_str = eq_str.replace("Nrm", "")
    
    beta = symbols("beta")
    rho = symbols("rho")
    syms = {"beta": beta, "CRRA": rho}
    
    lhs_str, rhs_str = eq_str.split("=")
    lhs = parse_expr(lhs_str.strip(), local_dict=syms)
    rhs = parse_expr(rhs_str.strip(), local_dict=syms)
    return Eq(lhs, rhs)


def parse_block(block):
    objective = parse_eq(block["equations"]["objective"][0])
    transitions = [parse_eq(eq) for eq in block["equations"]["transitions"]]
    definitions = [parse_eq(eq) for eq in block["equations"]["definitions"]]

    return objective, transitions, definitions


def subs_eq_lr(eq, substitutes):
    eq = eq.doit()
    # Substitute transitions and definitions into objective
    for subs_eq in substitutes:
        eq = eq.subs(subs_eq.lhs, subs_eq.rhs)

    return eq.doit()

def subs_eq_rl(eq, substitutes):
    eq = eq.doit()
    # Substitute transitions and definitions into objective
    for subs_eq in substitutes:
        eq = eq.subs(subs_eq.rhs, subs_eq.lhs)

    return eq.doit()

In [120]:
objective_eq, transitions_eqs, definitions_eqs = parse_block(deposit_block)

objective_eq = subs_eq_lr(objective_eq, transitions_eqs)
objective_eq = subs_eq_lr(objective_eq, definitions_eqs)

# Simplify the final equation
final_eq = simplify(objective_eq)

final_eq

Eq(v(m, n), w(-d + m, chi*log(d + 1) + d + n))

In [121]:
Eq(diff(final_eq.lhs, "m"), diff(final_eq.rhs, "m"))

Eq(Derivative(v(m, n), m), Subs(Derivative(w(_xi_1, chi*log(d + 1) + d + n), _xi_1), _xi_1, -d + m))

In [122]:
Eq(diff(final_eq.lhs, "n"), diff(final_eq.rhs, "n"))

Eq(Derivative(v(m, n), n), Subs(Derivative(w(-d + m, _xi_2), _xi_2), _xi_2, chi*log(d + 1) + d + n))

In [123]:
euler = Eq(0, diff(final_eq.rhs, "d"))
euler

Eq(0, (chi/(d + 1) + 1)*Subs(Derivative(w(-d + m, _xi_2), _xi_2), _xi_2, chi*log(d + 1) + d + n) - Subs(Derivative(w(_xi_1, chi*log(d + 1) + d + n), _xi_1), _xi_1, -d + m))

In [124]:
egm = solve(euler, "d")
solution = egm[0]
solution

(chi*Subs(Derivative(w(-d + m, _xi_2), _xi_2), _xi_2, chi*log(d + 1) + d + n) - Subs(Derivative(w(_xi_1, chi*log(d + 1) + d + n), _xi_1), _xi_1, -d + m) + Subs(Derivative(w(-d + m, _xi_2), _xi_2), _xi_2, chi*log(d + 1) + d + n))/(Subs(Derivative(w(_xi_1, chi*log(d + 1) + d + n), _xi_1), _xi_1, -d + m) - Subs(Derivative(w(-d + m, _xi_2), _xi_2), _xi_2, chi*log(d + 1) + d + n))

In [125]:
# needs to be run twice for some reason

solution = subs_eq_rl(solution, transitions_eqs)
solution = subs_eq_rl(solution, definitions_eqs)
solution = subs_eq_rl(solution, transitions_eqs)
solution = subs_eq_rl(solution, definitions_eqs)

In [126]:
Eq(Symbol("d"), simplify(solution))

Eq(d, (-chi*Derivative(w(l, b), b) - Derivative(w(l, b), b) + Derivative(w(l, b), l))/(Derivative(w(l, b), b) - Derivative(w(l, b), l)))

### Consumption Solution

In [127]:
objective_eq, transitions_eqs, definitions_eqs = parse_block(consumption_block)

objective_eq = subs_eq_lr(objective_eq, transitions_eqs)
objective_eq = subs_eq_lr(objective_eq, definitions_eqs)

# Simplify the final equation
final_eq = simplify(objective_eq)

final_eq

Eq(v(l, b), (DiscFac*LivPrb*(rho - 1)*w(-c + l, b) - c**(1 - rho))/(rho - 1))

In [128]:
Eq(diff(final_eq.lhs, "l"), diff(final_eq.rhs, "l"))

Eq(Derivative(v(l, b), l), DiscFac*LivPrb*Subs(Derivative(w(_xi_1, b), _xi_1), _xi_1, -c + l))

In [129]:
Eq(diff(final_eq.lhs, "b"), diff(final_eq.rhs, "b"))

Eq(Derivative(v(l, b), b), DiscFac*LivPrb*Derivative(w(-c + l, b), b))

In [130]:
euler = Eq(0, diff(final_eq.rhs, "c"))
euler

Eq(0, (-DiscFac*LivPrb*(rho - 1)*Subs(Derivative(w(_xi_1, b), _xi_1), _xi_1, -c + l) - c**(1 - rho)*(1 - rho)/c)/(rho - 1))

In [131]:
egm = solve(euler, "c")
solution = egm[0]
solution

(1/(DiscFac*LivPrb*Subs(Derivative(w(_xi_1, b), _xi_1), _xi_1, -c + l)))**(1/rho)

In [132]:
# needs to be run twice for some reason

solution = subs_eq_rl(solution, transitions_eqs)
solution = subs_eq_rl(solution, definitions_eqs)
solution = subs_eq_rl(solution, transitions_eqs)
solution = subs_eq_rl(solution, definitions_eqs)

In [133]:
Eq(Symbol("c"), simplify(solution))

Eq(c, (1/(beta*Derivative(w(a, b), a)))**(1/rho))